In [1]:
import os
import openai
from dotenv import load_dotenv


openai_api_key = os.getenv("OPENAI_API_KEY") 
openai.api_key = openai_api_key

# Search
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Using LlamaIndex as a Callable Tool

In [5]:
import logging

from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
from llama_index import Prompt, StorageContext, load_index_from_storage

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

qa_template = Prompt(
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Do not give me an answer if it is not mentioned in the context as a fact.\n"
    "Always state where the information was found, and if possible where more information can be accessed.\n"
    "Given this information, please provide me with an answer to the following question:\n"
    "\n{query_str}\n"
)

# LLM
llm = ChatOpenAI(temperature=0)

## Index tool from documents in vector store
storage_context = StorageContext.from_defaults(persist_dir="vector_db")
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine(
    streaming=True, text_qa_template=qa_template, similarity_top_k=5
)

## Websearch Tool
# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
)

# Search
search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search,
)

# chain
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)

tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(query_engine.query(q)),
        description="useful for when you want to answer questions about work and unemployment. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
    Tool(
        name="Websearch",
        func=lambda q: str(qa_chain({"question": q})),
        description="useful for when the LlamaIndex Tool does not have a sufficient answer in it's documents and you want to answer more general questions concerning laws and administration in Berlin, Germany. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

result = agent_executor.run(
    input="Where can I find information about the duties of landlords?"
)



> Entering new AgentExecutor chain...


INFO:langchain.retrievers.web_research:Generating questions for Google Search ...


Thought: Do I need to use a tool? Yes
Action: Websearch
Action Input: Duties of landlords in Berlin, Germany

INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Duties of landlords in Berlin, Germany', 'text': LineList(lines=['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. What are the responsibilities of landlords in Berlin, Germany?\n', '3. What are the duties and requirements for landlords in Berlin, Germany?'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. What are the responsibilities of landlords in Berlin, Germany?\n', '3. What are the duties and requirements for landlords in Berlin, Germany?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': 'Basic Law for the Federal Republic of Germany', 'link': 'https://www.gesetze-im-internet.de/englisch_gg/englisch_gg.html', 'snipp


Observation: {'question': 'Duties of landlords in Berlin, Germany', 'answer': 'The duties of landlords in Berlin, Germany include restrictions on rent increases and the obligation to provide compensation payments under certain circumstances. Rent increases are capped at a maximum of 20% within three years, or 15% in areas with a jeopardized housing supply. However, these restrictions do not apply if the rent-control scheme has ceased to apply or if the increase does not exceed the amount of the most recently payable compensation payment. Landlords may also demand information from tenants regarding compensation payments. These regulations are outlined in the German Civil Code (Bürgerliches Gesetzbuch). \n', 'sources': ''}


> Finished chain.


# Extend the Search and provide Sources

In [19]:
import os
import re
from typing import List

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
from pydantic import BaseModel, Field

# LLMChain
search_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search 
    results. Generate THREE Google search queries that are similar to
    this question. The output should be a numbered list of questions and each
    should have a question mark at the end: {question}""",
)


class LineList(BaseModel):
    """List of questions."""

    lines: List[str] = Field(description="Questions")


class QuestionListOutputParser(PydanticOutputParser):
    """Output parser for a list of numbered questions."""

    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = re.findall(r"\d+\..*?\n", text)
        return LineList(lines=lines)

# LLM
llm = ChatOpenAI(temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=search_prompt,
    output_parser=QuestionListOutputParser(),
)


# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
)

# Search
search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever_llm_chain = WebResearchRetriever(
    vectorstore=vectorstore,
    llm_chain=llm_chain,
    search=search,
)

def run_chain(user_input):
    docs = web_research_retriever_llm_chain.get_relevant_documents(user_input)
    chain = load_qa_chain(llm, chain_type="stuff")
    output = chain( {"input_documents": docs, "question": user_input}, return_only_outputs=True )
    return output["output_text"], docs


user_input = "Where can I find information about the duties of landlords in Berlin?"
output, docs = run_chain(user_input)

Fetching pages: 100%|##########| 2/2 [00:00<00:00, 20.26it/s]


In [28]:
docs
output

'You can find information about the duties of landlords in Berlin in Chapter 2, Subchapter 1, Section 556 of the Berlin Rent Act. This section specifically addresses agreements on operating costs and outlines the responsibilities of the owner or holder of the heritable building right. Additionally, you may also find relevant information in Subtitle 4, Section 581, which discusses contractual duties typical for a usufructuary lease.'

In [27]:
for doc in docs:
    page_content = doc.page_content
    metadata = doc.metadata
    source = metadata['source']
    title = metadata['title']
    
    # Process or use the extracted information as needed
    print(f"Source: {source}")
    print(f"Title: {title}")
    print(f"Page Content: {page_content}")
    print("-----")


Source: https://www.gesetze-im-internet.de/englisch_bgb/englisch_bgb.html
Title: German Civil Code BGB
Page Content: Chapter 2  
Rent

table of contents

Subchapter 1  
Agreements on rent

table of contents

Section 556  
Agreements on operating costs

(1) The contractual parties may agree that the lessee is to bear the operating
costs. Operating costs are the costs that are incurred on an ongoing basis by
the owner or the holder of the heritable building right as a result of the
ownership of or the heritable building right to the plot of land or as a
result of the use of the building, the outbuildings, facilities, installations
and the land in accordance with the purpose for which they are intended. The
drawing up of the statement of operating costs continues to be governed by the
Ordinance on Operating Costs ( _Betriebskostenverordnung_ ) of 25 November
2003 (Federal Law Gazette I pp. 2346, 2347). The Federal Government is
authorised to enact provisions on the drawing up of the state

# Agent with Tools - openai cookbook
https://cookbook.openai.com/examples/how_to_build_a_tool-using_agent_with_langchain

## imports

In [19]:
from typing import List, Union
import re
import openai
import pandas as pd
from langchain import LLMChain, OpenAI, SerpAPIWrapper

# Langchain imports
from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
)
from langchain.chains import RetrievalQA

# LLM wrapper
from langchain.chat_models import ChatOpenAI

# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings

# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
from langchain.vectorstores import Pinecone
from tqdm.auto import tqdm

## template with history

In [14]:
# Set up a prompt template which can interpolate the history
template_with_history = """You are BureauBot, a professional search engine who provides informative answers to users. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give detailed, informative answers

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

## template without history

In [26]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

## custom prompt template

In [27]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## custom output parser

In [28]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [29]:
llm = OpenAI(temperature=0)

## Index tool from documents in vector store
storage_context = StorageContext.from_defaults(persist_dir="vector_db")
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine(
    streaming=True, 
    text_qa_template=qa_template, 
    similarity_top_k=5
)

tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(query_engine.query(q)),
        description="useful for when you want to answer questions about work and unemployment. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
    Tool(
        name="Websearch",
        func=lambda q: str(qa_chain({"question": q})),
        description="useful for when you want to answer questions about work and unemployment and the LlamaIndex Tool does not have a sufficient answer in it's documents. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

# Re-initialize the agent with our new list of tools
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
multi_tool_names = [tool.name for tool in tools]
multi_tool_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=multi_tool_names
)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [30]:
multi_tool_memory = ConversationBufferWindowMemory(k=2)

multi_tool_executor = AgentExecutor.from_agent_and_tools(
    agent=multi_tool_agent, tools=tools, verbose=True, memory=multi_tool_memory
)

In [31]:
multi_tool_executor.run("Where can I find information about the duties of landlords?")



> Entering new AgentExecutor chain...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"

Thought: I should look for information about the duties of landlords
Action: Websearch
Action Input: duties of landlordsINFO:langchain.retrievers.web_research:Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4295 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

# Tools as Classes

In [ ]:
import logging
import os

from bot_utils import default_engine
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
from llama_index import Prompt
from llama_index.langchain_helpers.agents import IndexToolConfig, LlamaIndexTool
from langchain.tools import BaseTool
from langchain.agents import tool

# logging to see what sites and documents the web retriever is using
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

# Search
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

search = GoogleSearchAPIWrapper()
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")


class EngineTool:
    def __init__(self):
        
        self.qa_template = Prompt(
            "We have provided context information below. \n"
            "---------------------\n"
            "{context_str}"
            "\n---------------------\n"
            "Do not give me an answer if it is not mentioned in the context as a fact. \n"
            "Given this information, please provide me with an answer to the following question:\n{query_str}\n"
        )

        self.folder_with_index = "vector_db"
        self.number_top_results = 5

        self.default_engine = default_engine(
            self.folder_with_index, self.qa_template, self.number_top_results
        )

        # ERROR: IndexToolConfig awaits an instance of BaseQueryEngine
        self.tool_config = IndexToolConfig(
            query_engine=default_engine,
            name=f"Vector Index",
            description=f"Useful for when you want to answer queries about work and unemployment in Berlin, Germany",
            tool_kwargs={"return_direct": True},
        )

    def run(self):
        return LlamaIndexTool.from_tool_config(self.tool_config)


class RetrieverTool:
    """Tool to find information about work, unemployment, laws and adminstrative infromation in Berlin, Germany"""

    def __init__(self):
        # Initialize LLM
        self.llm = llm

        # Vectorstore
        self.vectorstore = Chroma(
            embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
        )

        # Initialize
        self.web_research_retriever = WebResearchRetriever.from_llm(
            vectorstore=self.vectorstore,
            llm=self.llm,
            search=search,
        )

    def run(self, user_input):
        qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
            self.llm, retriever=self.web_research_retriever
        )

        return qa_chain({"question": user_input})


class ToolChainAgent:
    def __init__(self):
        self.llm = llm

        # initialize tools
        self.q_engine_fct = EngineTool().run()
        self.retriever_fct = RetrieverTool(self.llm).run

        self.tools = [
            # embedding first
            Tool(
                name="query engine",
                description="use this tool first, to get anwsers about work and unemployment in Berlin, Germany",
                func=self.q_engine_fct,
            ),
            # web retriever goes through programmable search engine
            Tool(
                name="web_retriever",
                description="use this tool when the first tool failed and you want to answer questions about work and unempoyment in Berlin, Germany",
                func=self.retriever_fct,
            ),
        ]

        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
        )

    def run(self, user_input):
        """
        Agent runs query with the listed tools:

        1. query engine from custom embedding
        2. web retriever, which goes through programmable google-search

        Run the agent instance like this:
        agent = ToolChainAgent()
        agent.run(promt)
        """
        return self.agent(user_input)
